In [65]:
#second try on official python wrapper
import elasticsearch
from elasticsearch import Elasticsearch
from jsmin import jsmin
import os

In [2]:
%install_ext https://raw.githubusercontent.com/szeitlin/watermark/master/watermark.py

Installed watermark.py. To use it, type:
  %load_ext watermark


In [3]:
%load_ext watermark

In [4]:
%watermark -a "Samantha Zeitlin" -d -u -p elasticsearch,jsmin,python

Samantha Zeitlin 
Last updated: 04/13/2015 

elasticsearch 1.4.0
jsmin 2.1.1
python 2.7.9


In [2]:
es = Elasticsearch()

In [21]:
print dir(Elasticsearch)

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_bulk_body', 'abort_benchmark', 'benchmark', 'bulk', 'clear_scroll', 'count', 'count_percolate', 'create', 'delete', 'delete_by_query', 'delete_script', 'delete_template', 'exists', 'explain', 'get', 'get_script', 'get_source', 'get_template', 'index', 'info', 'list_benchmarks', 'mget', 'mlt', 'mpercolate', 'msearch', 'mtermvectors', 'percolate', 'ping', 'put_script', 'put_template', 'scroll', 'search', 'search_exists', 'search_shards', 'search_template', 'suggest', 'termvector', 'termvectors', 'update']


In [9]:
try:
    es.indices.create(index='official_test')
except elasticsearch.RequestError as re:
    print re

TransportError(400, u'IndexAlreadyExistsException[[official_test] already exists]')


In [104]:
current_map = es.indices.get_mapping(index='official_test', doc_type='notebook')
current_map

{u'official_test': {u'mappings': {u'notebook': {u'properties': {u'metadata': {u'properties': {u'name': {u'type': u'string'},
       u'signature': {u'type': u'string'}}},
     u'nbformat': {u'type': u'long'},
     u'nbformat_minor': {u'type': u'long'},
     u'worksheets': {u'properties': {u'cells': {u'properties': {u'cell_type': {u'type': u'string'},
         u'collapsed': {u'type': u'boolean'},
         u'input': {u'fields': {u'raw': {u'index': u'not_analyzed',
            u'type': u'string'}},
          u'type': u'string'},
         u'language': {u'type': u'string'},
         u'metadata': {u'type': u'object'},
         u'outputs': {u'include_in_parent': True,
          u'properties': {u'html': {u'type': u'string'},
           u'metadata': {u'type': u'object'},
           u'output_type': {u'type': u'string'},
           u'png': {u'type': u'binary'},
           u'prompt_number': {u'type': u'long'},
           u'stream': {u'type': u'string'},
           u'text': {u'type': u'string'}},
  

In [18]:
#note: elasticsearch does not have helpers that I can see, but it does have both logger and logging 

In [103]:
with open('mapping_official.json', 'r') as map:
    body = jsmin(map.read())# + '\n'
    
    #the trick is that the mapping starts at the level of the document type, not the index, and doesn't need the word 
    #'mappings' in it, as was shown in some (misleading!) examples
    
    try:
        es.indices.put_mapping(index ='official_test',doc_type='notebook',body=body)
    except elasticsearch.RequestError as re:
        print re

In [97]:
print es.indices.put_mapping.__doc__


        Register specific mapping definition for a specific type.
        `<http://www.elasticsearch.org/guide/en/elasticsearch/reference/current/indices-put-mapping.html>`_

        :arg index: A comma-separated list of index names the mapping should be
            added to (supports wildcards); use `_all` or omit to add the
            mapping on all indices.
        :arg doc_type: The name of the document type
        :arg body: The mapping definition
        :arg allow_no_indices: Whether to ignore if a wildcard indices
            expression resolves into no concrete indices. (This includes `_all`
            string or when no indices have been specified)
        :arg expand_wildcards: Whether to expand wildcard expression to concrete
            indices that are open, closed or both., default u'open'
        :arg ignore_conflicts: Specify whether to ignore conflicts while
            updating the mapping (default: false)
        :arg ignore_unavailable: Whether specified concret

In [11]:
path = '/Users/szeitlin/mystuff/projects/nbindex/nbindex/testdir'

prefix = '{"index": {"_index": "official_map", "_type":"notebook"}}\n'

newline = '\n'

with open('test_temp.json', 'wb') as body:
    for eachfile in os.listdir(path):
        fullname = os.path.join(path,eachfile)
        with open(fullname,'r') as each:
            body.write(prefix)
            body.write(jsmin(each.read()) + newline)
            

In [14]:
with open('test_temp.json', 'r') as body:
    obj = body.read()
    es.bulk(obj, index='test3', doc_type='notebook')

In [34]:
print es.cat.help()

=^.^=
/_cat/allocation
/_cat/shards
/_cat/shards/{index}
/_cat/master
/_cat/nodes
/_cat/indices
/_cat/indices/{index}
/_cat/segments
/_cat/segments/{index}
/_cat/count
/_cat/count/{index}
/_cat/recovery
/_cat/recovery/{index}
/_cat/health
/_cat/pending_tasks
/_cat/aliases
/_cat/aliases/{alias}
/_cat/thread_pool
/_cat/plugins
/_cat/fielddata
/_cat/fielddata/{fields}



In [40]:
print es.cat.indices()

green  open official_test 1 0 0 0   115b   115b 
green  open test2         1 0 0 0   115b   115b 
yellow open mapping       5 1 0 0   575b   575b 
yellow open .kibana       1 1 4 0 13.9kb 13.9kb 
green  open test          1 0 0 0   115b   115b 
yellow open notebooks     5 1 0 0   575b   575b 
yellow open {             5 1 0 0   575b   575b 
green  open {mappings:    1 0 0 0   115b   115b 
green  open test3         1 0 3 0 75.7kb 75.7kb 

